# Recommender Systems 2022/2023

## Practice Session 11 - MF with PyTorch

PyTorch, Tensorflow, Keras are useful framework that allow you to build machine learning models (from linear regression to complex deep learning methods) and hide almost all of the complexity related to the training. Usually, you only have to create an object that starting from the model parameters will be able to compute your prediction, then specify the loss and the framework automatically calculates the gradients.

#### Performance warning!
In image processing tasks one usually has an image, maybe a reasonably large one (1000x1000x3, hence 3\*10^6 data points) on which a complex network is applied (multiple convolution operations, pooling etc). The computationally expensive part can be effectively parallelized by the framework and hence the speedup over using single-core operations is massive.

Unfortunately here we are not dealing with images, but with user profiles. In terms of data this means each profile can be in the 10^5 - 10^6 items. The issue arises when considering the model. If you use a matrix factorization model, the core of the operation is a dot product between two embedding vectors, which is an extremely fast operation. There is hardly any speedup and the burden of the overall infrastructure is not offset by it. For this reason, if you use a profiler you will see that 80-90% of the time is spent in the data sampling phase (because it is done in python it can be quite slow) and the actual prediction computation is a tiny fraction of the time. Overall, a *simple* matrix factorization model may be 10x slower if implemented with pytorch without being careful. You must ensure to use a fast sampling script (Cython here again) and a reasonably powerful GPU.

#### Prototyping
Given how the complexity of gradients and such is hidden, pytorch becomes a great tool for prototyping. It is very easy to change someting in your model becayse you do not need to dig in Cython code. For example, you may implement a SLIM MSE method that uses as inital parameters the similarity computed with an item-based cosine method, or you may create a hybrid of multiple similarities and lear the weights to use for each similarity.

## What do we need

* A Dataset object to load the data
* Model object
* Training loop

In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: Movielens10M
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10126, feature occurrences: 128384, density 1.19E-03
	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10106, feature occurrences: 106820, density 9.90E-04
	ICM name: ICM_year, Value range: 6.00E+00 / 2.01E+03, Num features: 1, feature occurrences: 10681, density 1.00E+00




### MF models rely upon latent factors for users and items which are called 'embeddings'

![latent factors](https://miro.medium.com/max/988/1*tiF4e4Y-wVH732_6TbJVmQ.png)

In [2]:
num_factors = 10

n_users, n_items = URM_train.shape

In [3]:
import torch

# Creates U
user_factors = torch.nn.Embedding(num_embeddings=n_users, embedding_dim=num_factors)

# Creates V
item_factors = torch.nn.Embedding(num_embeddings=n_items, embedding_dim=num_factors)

In [4]:
user_factors

Embedding(69878, 10)

In [5]:
item_factors

Embedding(10681, 10)

## In order to compute the prediction you have to:
- Get a list of user and item indices (as tensors)
- Get the user and item embedding
- Compute the element-wise product of the embeddings

In [6]:
user_index = torch.Tensor([42]).type(torch.LongTensor)
item_index = torch.Tensor([42]).type(torch.LongTensor)

user_index, item_index

(tensor([42]), tensor([42]))

### Notice that each object has a "grad_fn=..." attribute, which si going to be used for the automatic gradient compuation to go backwards in the operations required to compute the prediction.

In [7]:
current_user_factors = user_factors(user_index)
current_item_factors = item_factors(item_index)

current_user_factors, current_item_factors

(tensor([[-0.4088,  0.8317,  1.7737, -0.0576,  1.3666, -0.5248, -0.8404, -1.2425,
          -0.5362, -0.2207]], grad_fn=<EmbeddingBackward0>),
 tensor([[-0.3708,  0.5689, -0.5090, -0.4367, -1.1786,  0.8405, -1.5139,  1.2917,
          -1.4407, -0.7400]], grad_fn=<EmbeddingBackward0>))

Now the dot product is just a summation over the elementwise product. 

In [8]:
prediction = torch.mul(current_user_factors, current_item_factors).sum()
prediction

tensor(-1.7016, grad_fn=<SumBackward0>)

Notice how the "grad_fn" states "SubBackward", the prediction was indeed due to a sum

#### We can also use the einstein summation format, which is particularly useful when you have a more complex equation to compute the prediction

The einstein summation allows you to write the prediction in terms of the indices of a summation. In this case we want to iterate both embedding vectors, perform an element-by-element product and then sum at the end. Be careful on the dimensions, in this case the factors have two dimensions (the row dimension is 1 so in practice it is useless). We use "b" to iterate over the rows (useful when we compute batches of predictions to parallelize) and "i" is the latent factor index.

$$ r_b = \sum_{i}U'_{bi}V'_{bi} $$

In [9]:
torch.einsum("bi,bi->b", current_user_factors, current_item_factors)

tensor([-1.7016], grad_fn=<ViewBackward0>)

### To take the result of the prediction and transform it into a traditional numpy array you have to:
- call .detach() to disconnect the tensor from the automatic gradient tracking
- then .numpy()

### The result is an array of 1 cell

In [10]:
prediction_numpy = prediction.detach().numpy()
print("Prediction is {:.2f}".format(prediction_numpy))

Prediction is -1.70


# Train a MF MSE model with PyTorch

# Step 1 Create a Model python object

### The model should implement the forward function which computes the prediction as we did before

In [11]:
class MF_MSE_PyTorch_model(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors):
        super(MF_MSE_PyTorch_model, self).__init__()

        self.n_users = n_users
        self.n_items = n_items

        self.user_factors = torch.nn.Embedding(num_embeddings=self.n_users, embedding_dim=n_factors)
        self.item_factors = torch.nn.Embedding(num_embeddings=self.n_items, embedding_dim=n_factors)

    def forward(self, user_batch, item_batch):
        user_factors_batch = self.user_factors(user_batch)
        item_factors_batch = self.item_factors(item_batch)

        prediction_batch = torch.mul(user_factors_batch, item_factors_batch).sum()

        return prediction_batch

    def get_W(self):
        return self.user_factors.weight.detach().cpu().numpy()

    def get_H(self):
        return self.item_factors.weight.detach().cpu().numpy()

# Step 2 Setup PyTorch devices and Data Reader

In [12]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("MF_MSE_PyTorch: Using CUDA")
else:
    device = torch.device('cpu')
    print("MF_MSE_PyTorch: Using CPU")

MF_MSE_PyTorch: Using CUDA


### Create an instance of the model and specify the device it should run on

In [13]:
model = MF_MSE_PyTorch_model(n_users, n_items, num_factors).to(device)

### Choose loss functions (Mean Squared Error in our case), there are quite a few to choose from

In [14]:
lossFunction = torch.nn.MSELoss(reduction="sum")

Alternatively one can implement it 

In [15]:
def _my_MSE_loss(model, user, item):
    
    # Compute prediction for each element in batch
    prediction = model.forward(user, item)

    # Compute total loss for batch
    loss = (prediction - rating).pow(2).mean()
    
    return loss

### Select the optimizer to be used for the model parameters: Adam, AdaGrad, RMSProp etc... 

In [16]:
learning_rate = 1e-4
l2_reg = 1e-3

optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay = l2_reg*learning_rate)

### Define the DatasetInteraction, which will be used to load a specific data point

A DatasetInteraction will implement the Dataset class and provide the \_\_getitem\_\_(self, index) method, which allows to get the data points indexed by that index.

Since we need the data to be a tensor, we pre inizialize everything as a tensor. In practice we save the URM in coordinate format (user, item, rating)

In [17]:
from torch.utils.data import Dataset
import numpy as np

class DatasetInteraction(Dataset):
    def __init__(self, URM):

        URM = URM.tocoo()
        self.n_data_points = URM.nnz

        self._row = torch.tensor(URM.row).type(torch.LongTensor)
        self._col = torch.tensor(URM.col).type(torch.LongTensor)
        self._data = torch.tensor(URM.data).type(torch.FloatTensor)
       
    def __getitem__(self, index):
        return self._row[index], self._col[index], self._data[index]


    def __len__(self):
        return self.n_data_points


### We pass the DatasetIterator to a DataLoader object which manages the use of batches and so on...

In [18]:
from torch.utils.data import DataLoader

# A large batch_size (256, 512...) improves parallelization, but the gradient becomes more smoot
# at some point the performance will increase but at the expense of the final prediction quality
batch_size = 64

dataset_iterator = DatasetInteraction(URM_train[0:1000])

train_data_loader = DataLoader(dataset=dataset_iterator,
                               batch_size=batch_size,
                               shuffle=True,
                               #num_workers = 2,
                              )

## And now we ran the usual epoch steps
* Data point sampling
* Prediction computation
* Loss function computation
* Gradient computation
* Update

In [19]:
batch = next(iter(train_data_loader))
batch

[tensor([417, 173,  67, 442, 655, 821, 507, 486, 842, 507, 279, 591, 820, 243,
         284, 405, 600, 855, 891, 139, 201, 233, 465, 233, 746, 604, 729, 111,
         841, 551, 326, 717, 456, 422, 377, 451, 300, 929, 820, 379, 573, 405,
         940, 605, 422, 407, 731, 280, 764, 828, 350, 758, 825, 402, 507, 377,
         446, 823, 781, 562, 406, 770, 781, 862]),
 tensor([ 417, 1377,  331, 2901,  691, 1055, 3040, 1864,  528,   96, 1994,  691,
           82,  213,  367, 1864, 1429, 1288,   16,  799,  466, 3718,   88,   36,
         1306, 1040,  811, 1339, 1117,  563,  192, 1376, 1419, 1622, 1008,   31,
           39,  302,   72, 1603,  368,  204,  613, 3071, 1139,  300,  113,  411,
         1076, 1296,    4,  198, 2015, 1429, 2847, 2192, 1134,  437,   63,   41,
          498,  177, 4109,  573]),
 tensor([4.0000, 4.0000, 4.0000, 3.5000, 4.0000, 2.0000, 4.0000, 3.5000, 1.0000,
         5.0000, 4.0000, 2.0000, 2.0000, 4.5000, 4.0000, 2.0000, 5.0000, 4.0000,
         4.0000, 4.0000, 5.0000

In [20]:
%%time
from tqdm.notebook import tqdm

def run_epoch(data_iterator):
    
    epoch_loss = 0
    for batch in tqdm(data_iterator):

        # Clear previously computed gradients
        optimizer.zero_grad()

        user, item, rating = batch
        user = user.to(device)
        item = item.to(device)
        rating = rating.to(device)

        # Compute prediction for each element in batch
        prediction = model.forward(user, item)

        # Compute total loss for batch
        loss = (prediction - rating).pow(2).mean()

        # Compute gradients given current loss
        loss.backward()

        # Apply gradient using the selected optimizer
        optimizer.step()

        epoch_loss += loss.item()
        
        
run_epoch(train_data_loader)

  0%|          | 0/1661 [00:00<?, ?it/s]

CPU times: total: 4.41 s
Wall time: 3.72 s


## After the train is complete (after many epochs), we can get the matrices in the usual numpy format

In [21]:
user_factors = model.get_W()
item_factors = model.get_H()

In [22]:
user_factors, user_factors.shape

(array([[ 0.95083886, -1.2302514 , -0.9758749 , ...,  0.43588272,
         -1.2674538 , -1.889634  ],
        [-0.9309265 ,  0.27153483,  0.50834876, ..., -1.0890548 ,
         -0.06128926, -1.1721663 ],
        [-0.29489905, -1.9374983 ,  1.8267319 , ..., -1.3253108 ,
         -0.69661677, -0.9291013 ],
        ...,
        [ 0.00301835,  0.18262248, -0.03628764, ...,  0.21849915,
          1.1100464 ,  0.07815299],
        [-1.0627358 ,  0.2809174 , -0.8795823 , ..., -0.03074418,
         -0.23348367, -2.0146549 ],
        [ 0.8826899 , -1.3980063 ,  1.2482066 , ...,  1.8058219 ,
         -0.33894625,  0.02159044]], dtype=float32),
 (69878, 10))

In [23]:
item_factors, item_factors.shape

(array([[ 1.71100342e+00, -1.71951842e+00,  8.17174241e-02, ...,
         -1.48534536e+00, -9.88895446e-02, -4.12131786e-01],
        [ 1.06533170e+00,  8.93806458e-01,  2.44553253e-01, ...,
          5.43021798e-01,  4.13350135e-01, -8.92155409e-01],
        [ 9.41068828e-01, -5.04354715e-01,  5.77265263e-01, ...,
          1.09992906e-01,  1.11340630e+00, -2.55454421e-01],
        ...,
        [ 1.23278880e+00,  1.24008727e+00,  1.24930787e+00, ...,
         -2.73840427e-01,  7.57642210e-01,  1.29161060e+00],
        [-1.34193742e+00,  1.46697676e+00, -1.98895597e+00, ...,
          1.57310751e-05,  2.72938877e-01,  2.28593826e+00],
        [-9.53395128e-01,  1.13207646e-01, -2.48263434e-01, ...,
         -2.05180216e+00, -2.29296136e+00, -7.50487745e-01]], dtype=float32),
 (10681, 10))

### Let's use the profile to monitor which operation is taking the most time

The following disables code output line wrapping to better read the profiler lor

In [24]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [25]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], 
             profile_memory=False, record_shapes=False) as prof:

    run_epoch(train_data_loader)
    
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))
# prof.export_chrome_trace("trace.json")    

  0%|          | 0/1661 [00:00<?, ?it/s]

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        21.53%        2.470s        65.54%        7.517s       4.523ms     468.773ms         3.78%        6.594s       3.967ms          1662  
                                           aten::select        15.14%        1.736s        15.68%        1.798s       5.639us        2.092s        16.88%        2.504s       7.852us        318855  
         

Notice how approximately 65\% of the CPU time is spent by:
* SingleProcessDataLoader
* aten::select
* aten::stack
* aten::unsqueeze

All of these are either exclusively or mostly related to the data loading and their type conversion to tensors.

Only a few percent of the time is spent by the optmizer and actual embedding product. This indicates our model is terribly inefficient.

### A less terribly inefficient data sampling

Despite following the strategy recommended by PyTorch, the previous data sampler is terribly slow likely resulting in 80-90% of the global training time of the model to be wasted waiting for the sampling process. This is partially because the *DataLoader* object calls your custom *Dataset* object to get each single data point and then concatenates them as tensors, making lots of intermediate calls and conversions.

You can check this with the simple profiler PyTorch provides.

I STRONGLY recommend to use a custom sampler that leverages some basic vectorization to load a full batch of data instead. For example as follows

In [26]:
from torch.utils.data import Dataset
import math
import numpy as np
import scipy.sparse as sps

class InteractionIterator(object):
    """
    This Sampler samples among all the existing user-item interactions *uniformly at random*:
    - One of the interactions in the dataset is sampled

    The sample is: user_id, item_id, rating
    """

    def __init__(self, URM_train, batch_size = 1):
        super(InteractionIterator, self).__init__()

        self.URM_train = sps.coo_matrix(URM_train)
        self.n_users, self.n_items = self.URM_train.shape
        self.n_data_points = self.URM_train.nnz
        self.n_sampled_points = 0
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(self.n_data_points/self.batch_size)

    def __iter__(self):
        self.n_sampled_points = 0
        return self

    def __next__(self):

        if self.n_sampled_points >= self.n_data_points:
            raise StopIteration

        this_batch_size = min(self.batch_size, self.n_data_points-self.n_sampled_points)
        self.n_sampled_points += this_batch_size

        index_batch = np.random.randint(self.n_data_points, size = this_batch_size)

        return torch.from_numpy(self.URM_train.row[index_batch]).long(),\
               torch.from_numpy(self.URM_train.col[index_batch]).long(), \
               torch.from_numpy(self.URM_train.data[index_batch]).float()


In [27]:
batch_train_data_loader = InteractionIterator(URM_train[0:1000], batch_size=batch_size)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], 
             profile_memory=False, record_shapes=False) as prof:

    run_epoch(batch_train_data_loader)
    
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))
# prof.export_chrome_trace("trace.json") 

  0%|          | 0/1661 [00:00<?, ?it/s]

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                              aten::_local_scalar_dense        16.08%     541.972ms        16.08%     541.972ms     108.764us     800.528ms        18.51%     800.528ms     160.652us          4983  
                                            aten::copy_        14.86%     500.583ms        14.86%     500.583ms      33.486us        1.355s        31.33%        1.355s      90.642us         14949  
         

Much better! We have removed the data bottleneck and the training proceeds remarkably faster!

Notice how we still have some data related processing 
* aten::copy_
* aten::_local_scalar_dense 
* aten::_to_copy

These are mostly related to the CPU-GPU data transfer and are still significant

### What if I want to change the sampling?
The DatasetInteraction can be modified to obtain the desired behaviour, for example adding some negative (zero-rated) items in the sampling. If we want our model to be able to distinguish between positive and negative items we need to let the model see negative data as well, in our case the negative data is the zero-rated items.

In [28]:
class DatasetInteraction(Dataset):
    def __init__(self, URM_train, positive_quota):
        
        self._URM_train = sps.csr_matrix(URM_train)
        
        URM_train = URM_train.tocoo()
        self.n_data_points = URM.nnz

        self._row = torch.tensor(URM_train.row).type(torch.LongTensor)
        self._col = torch.tensor(URM_train.col).type(torch.LongTensor)
        self._data = torch.tensor(URM_train.data).type(torch.FloatTensor)
        self._positive_quota = positive_quota
       
    def __getitem__(self, index):
        select_positive_flag = torch.rand(1, requires_grad=False) > self._positive_quota

        if select_positive_flag[0]:
            return self._row[index], self._col[index], self._data[index]
        else:
            user_id = self._row[index]
            seen_items = self._URM_train.indices[self._URM_train.indptr[user_id]:self._URM_train.indptr[user_id+1]]
            negative_selected = False

            while not negative_selected:
                negative_candidate = torch.randint(low=0, high=self.n_items, size=(1,))[0]

                if negative_candidate not in seen_items:
                    item_negative = negative_candidate
                    negative_selected = True

            return self._row[index], item_negative, torch.tensor(0.0)

        
        return self._row[index], self._col[index], self._data[index]


    def __len__(self):
        return self.n_data_points

You may also change the dataset iterator to one that samples the user profile rather than the specific interaction

In [29]:
class UserProfile_Dataset(Dataset):
    def __init__(self, URM_train, device):
        super().__init__()
        URM_train = sps.csr_matrix(URM_train)
        self.device = device

        self.n_users, self.n_items = URM_train.shape
        self._indptr = URM_train.indptr
        self._indices = torch.tensor(URM_train.indices, dtype = torch.long, device=device)
        self._data = torch.tensor(URM_train.data, dtype = torch.float, device=device)

    def __len__(self):
        return self.n_users

    def __getitem__(self, user_id):
        start_pos = self._indptr[user_id]
        end_pos = self._indptr[user_id+1]

        user_profile = torch.zeros(self.n_items, dtype=torch.float, requires_grad=False, device=self.device)
        user_profile[self._indices[start_pos:end_pos]] = self._data[start_pos:end_pos]

        return user_profile

### What if I want to implement AsySVD? SLIM EN ... 
You just have to change the pytorch model with the desired one (easy to do)

Note these two models work by sampling the whole user profile, they can be adapted to a sampler that provides single interactions

In [30]:
from torch import nn

class AsySVDModel(nn.Module):

    def __init__(self, embedding_size = None, n_items = None, device = None):
        super().__init__()

        self._embedding_item_1 = torch.nn.Parameter(torch.randn((n_items, embedding_size)))
        self._embedding_item_2 = torch.nn.Parameter(torch.randn((embedding_size, n_items)))

    def forward(self, user_profile_batch):
        # input shape is batch_size x n items
        # r_hat_bi = SUM{e=0}{e=embedding_size} SUM{j=0}{j=n items} r_bj * V1_je * V2_ei
        layer_output = torch.einsum("bj,je,ei->bi", user_profile_batch, self._embedding_item_1, self._embedding_item_2)
        return layer_output

In [31]:
class SDenseModel(nn.Module):

    def __init__(self, n_items = None, device = None):
        super().__init__()

        self._S = torch.nn.Parameter(torch.zeros((n_items, n_items)))

    def forward(self, user_profile_batch):
        # input shape is batch_size x n items
        # r_hat_bi = SUM{j=0}{j=n items} r_bj * S_ji
        layer_output = torch.einsum("bj,ji->bi", user_profile_batch, self._S)
        return layer_output

### What if I want to change the loss function?
You can just implement the new one, BPR is quite simple. Make sure that the dataset iterator samples the right data

In [32]:
class BPR_Dataset(Dataset):
    def __init__(self, URM_train):
        super().__init__()
        self._URM_train = sps.csr_matrix(URM_train)
        self.n_users, self.n_items = self._URM_train.shape

    def __len__(self):
        return self.n_users

    def __getitem__(self, user_id):

        seen_items = self._URM_train.indices[self._URM_train.indptr[user_id]:self._URM_train.indptr[user_id+1]]
        item_positive = np.random.choice(seen_items)

        negative_selected = False

        while not negative_selected:
            negative_candidate = np.random.randint(low=0, high=self.n_items, size=1)[0]

            if negative_candidate not in seen_items:
                item_negative = negative_candidate
                negative_selected = True

        return user_id, item_positive, item_negative


In [33]:
def loss_BPR(model, batch):
    user, item_positive, item_negative = batch

    # Compute prediction for each element in batch
    x_ij = model.forward(user, item_positive) - model.forward(user, item_negative)

    # Compute total loss for batch
    loss = -x_ij.sigmoid().log().mean()

    return loss